In [1]:
# convert *.json inputs into *.csv file for checking purposes

In [2]:
import numpy as numpy
import pandas as pd
import json
import os

In [3]:
cwd = os.getcwd()

file_to_check = "raw_data/HandoverOOData_ICE_PO_20240117.json"
file_path = cwd + '/' + file_to_check

In [4]:
with open(file_path,'r') as f:
    try:
        obj = json.load(f)
    except:
        raise NameError('Erreur à la lecture du *.json')

In [5]:
obj = obj.get('cfe')

In [6]:
obj

[{'cpqID': '5356f09c-ee4b-4ef9-bfeb-abff4b53b4d6',
  'cfeExternalId': 'T2-SML',
  'name': 'Small Configuration',
  'contractCaseId': '22.NO.303971.CC',
  'quoteId': '22.NO.303971.CC.CQ.01',
  'extQuoteRefPOref': '2000002905',
  'orderIntakePolicy': 'O2E Order Intake',
  'cfeType': 'TYPE 2',
  'cfeInvoiceable': True,
  'salesCurrency': 'EUR',
  'incoterm': 'DDP',
  'incotermNamedPlace': None,
  'taxClassification': None,
  'invoice': {'type': 'Event-based',
   'billingPlan': [{'billingMilestone': 'BM#1-HW',
     'billingMilestoneType': 'Delivery',
     'billingMilestoneValue': '100%'}]},
  'payment': {'termMetric': 'D', 'dueFrom': 'Date of Invoice', 'term': '60'},
  'pricePeriod': {'name': 'Phase 1',
   'startDate': '2023-07-01',
   'endDate': '2024-06-30'},
  'CLP': '35896.08',
  'CP': '16153.24',
  'qty': '16',
  'pricingMetric': 'Unit',
  'pricingFactor': '1',
  'pricingConditionType': 'Per unit',
  'complexPricing': False,
  'orderableConfig': True,
  'orderPrepStatus': 'Single DQ',

In [7]:
columns_df = [
    'cfeType',
    'cfeExternalId',
    'name',
    'CLP',
    'CP',
    'qty',
    'profitCenter',
    'eqCLP',
    'eqCP',
    'code',
    'description',
    'qty_si',
    'eqCLP_si',
    'eqCP_si'
]

data_for_df = []  # chaque entrée de la liste est une liste des 14 valeurs d'une ligne

df = pd.DataFrame(columns=columns_df)

In [8]:
# format_json = {
#     'cfe' : {
#         'cfeExternalId' : 'item',
#         'name' : 'item',
#         'cfeType' : 'item',
#         'cfeT1' : {
#             'cfeExternalID' : 'item',
#             'cfeType' : 'item',
#             'name' : 'item',
#             'CLP' : 'item',
#             'CP' : 'item',
#             'qty' : 'item',
#             'pce' : {
#                 'profitCenter' : 'item',
#                 'eqCLP' : 'item',
#                 'eqCP' : 'item',
#                 'si' : {
#                     'code' : 'item',
#                     'description' : 'item',
#                     'qty' : 'item'
#                     'eqCLP' : 'item',
#                     'eqCP' : 'item'
#                 }
#             }
#             }
#     }
# }

In [9]:
labels_l1 = [
    'cfeType',
    'cfeExternalId',
    'name'
]

labels_l2 = [
    'cfeType',
    'cfeExternalId',
    'name',
    'CLP',
    'CP',
    'qty'
]

labels_l3 = [
    'profitCenter',
    'eqCLP',
    'eqCP'
]

labels_l4 = {
    'code' : 'code',
    'description' : 'description',
    'qty_si' : 'qty',
    'eqCLP_si' : 'eqCLP',
    'eqCP_si' : 'eqCP'
}

In [10]:
for dict1 in obj:
    # ---- 1er niveau -------------------------------------
    ligne_sup = {}
    for l in labels_l1:
        ligne_sup[l] = [dict1.get(l)]
    df_sup = pd.DataFrame(ligne_sup, columns=columns_df)
    df = pd.concat([df, df_sup], axis=0)
    # ---- 2e niveau --------------------------------------------
    list2 = dict1.get('cfeT1')
    for d2 in list2:
        ligne_sup = {}
        for l2 in labels_l2:
            ligne_sup[l2] = [d2.get(l2)]
        df_sup = pd.DataFrame(ligne_sup, columns=columns_df)
        df = pd.concat([df, df_sup], axis=0)
        # -- 3e niveau : PCE ---------------------------
        list3 = d2.get('pce')
        for d3 in list3:
            ligne_sup = {}
            for l3 in labels_l3:
                ligne_sup[l3] = [d3.get(l3)]
            ligne_sup['cfeType'] = ['PCE']
            df_sup = pd.DataFrame(ligne_sup, columns=columns_df)
            df = pd.concat([df, df_sup], axis=0)
            # -- 4e niveau : SI ----------------------------
            list4 = d3.get('si')
            if list4 is not None and len(list4)>=1:
                for d4 in list4:
                    ligne_sup = {}
                    for k,v in labels_l4.items():
                        ligne_sup[k] = [d4.get(v)]
                    ligne_sup['cfeType'] = ['SI']
                    df_sup = pd.DataFrame(ligne_sup, columns=columns_df)
                    df = pd.concat([df, df_sup], axis=0)

In [11]:
df

,cfeType,cfeExternalId,name,CLP,CP,qty,profitCenter,eqCLP,eqCP,code,description,qty_si,eqCLP_si,eqCP_si
0,TYPE 2,T2-SML,Small Configuration,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,TYPE 1,475266B,ABIO AirScale Capacity,7736.23,3481.30,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,PCE,NaN,NaN,NaN,NaN,NaN,PC00006831,7736.23,3481.30,NaN,NaN,NaN,NaN,NaN
0,SI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,475266B,ABIO AirScale Capacity,1.0,7736.23,3481.30
0,TYPE 1,473764A,ASIB AirScale Common,1742.23,784.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,PCE,NaN,NaN,NaN,NaN,NaN,PC00006831,653.34,0.0,NaN,NaN,NaN,NaN,NaN
0,SI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,473098A,AMIA AirScale Subrack,1.0,653.34,0.0
0,TYPE 1,SR000035.T,ABIO Baseband Capacity Activation LTU,362.97,0.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,PCE,NaN,NaN,NaN,NaN,NaN,PC00006834,362.97,0.0,NaN,NaN,NaN,NaN,NaN


In [12]:
file_output = cwd + '/output/' + 'output.csv'

with open(file_output, 'w') as f:
    df.to_csv(f, index=None)